In [ ]:
import traitlets
import os
from jetracer.default_racecar import DefaultRacecar
from jetcam.utils import bgr8_to_jpeg
from jetcam.imx219 import IMX219
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

In [ ]:
camera = IMX219(capture_width=1280, capture_height=720, fps=30, width=224, height=224)

In [ ]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='steering')
throttle_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='throttle')

def display_steering(camera_image):
    image = np.copy(camera_image)
    steering = steering_slider.value
    throttle = throttle_slider.value
    x = int(steering * 224 / 2 + 112)
    y = int(throttle * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_steering)

display(widgets.HBox([image_widget, target_widget]), steering_slider, throttle_slider)

In [ ]:
controller = widgets.Controller(index=2)

display(controller)

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (steering_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (throttle_slider, 'value'))

In [ ]:
def steering_throttle_uuid(steering, throttle):
    steering_value = steering * 50 + 50
    throttle_value = throttle * 50 + 50
    return 'xy_%03d_%03d_%s' % (steering_value, throttle_value, uuid1())

In [ ]:
DATASET_DIR = 'dataset_xy_circuitlaunch'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def save_steering(change):
    if change['new']:
        uuid = steering_throttle_uuid(steering_slider.value, throttle_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[13].observe(save_steering, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

In [ ]:
!zip -r -q circuitlaunch_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}